## Read .ghg metadata

In [65]:
import os
import re
import time
import zipfile
from datetime import datetime
import configparser
import pandas as pd
import numpy as np
from io import TextIOWrapper

from ipywidgets import FloatProgress
from IPython.display import display

def Parse_Cal(key,string):
    # Get Calibration metadata from co2app file
    Cal_Info = re.search(r''+key+'(.*?)\)\)', string).group(1)
    Cal_Date = Cal_Info.split('(Date ')[-1].replace(' at ',' ')
    Cal_Stats = Cal_Info.split('(Date ')[0]
    Cal_Stats = re.search(r'\((.*)\)', Cal_Stats).group(1).replace(')(',' ')
    Cal_Stats = pd.DataFrame(np.array([v for v in Cal_Stats.split(' ')]).reshape(-1,2))
    Cal_Stats = Cal_Stats.rename(columns={0:'Attribute',1:'Value'})
    Cal_Stats['Attribute']=key+'_'+Cal_Stats['Attribute'].astype(str)
    try:
        timestamp = datetime.strptime(Cal_Date,'%b %d %Y %H:%M:%S')
    except:
        timestamp = pd.Timestamp('nat')
        pass
    Cal_Stats = pd.concat(
        [Cal_Stats,pd.DataFrame(data={'Attribute':[key+'_Timestamp'],'Value':[timestamp]},index=[0])],
        axis=0,
        ignore_index=True
    )
    return(Cal_Stats)

def Parse_Coef(key,string):
    # Get Calibration coefficients from co2app file
    Coef_Info = re.search(r''+key+'\((.*?)\)\)', string).group(1).replace(')(',' ')
    Coef_Info = pd.DataFrame(np.array([v for v in Coef_Info.split(' ')]).reshape(-1,2))
    Coef_Info = Coef_Info.rename(columns={0:'Attribute',1:'Value'})
    Coef_Info['Attribute']=key.replace(' ','')+'_'+Coef_Info['Attribute'].astype(str)
    return(Coef_Info)

def Read_co2app(co2app):
    # Parse the system_config\co2app.conf file to get calibration settings
    # Only an option for co2/h2o for now
    # Does not appear the ch4 calibration values are saved anywhere?
    Coef = re.search(r'Coef(.*?)\)\)\)', co2app).group(0)
    Coef_Keys =  ['CO2 ','H2O ','Pressure ','MaxRef']
    Coef_Summary = pd.concat(
        [Parse_Coef(Val,Coef) for Val in Coef_Keys],
        axis=0,
        ignore_index=True)#.set_index('Key')

    Calibrate = re.search(r'Calibrate(.*?)\)\)\)', co2app).group(0)
    Calibrate_Keys =  ['ZeroCO2','SpanCO2','ZeroH2O','SpanH2O'] # Span2CO2 & Span2H2O not needed
    Calibrate_Summary = pd.concat(
        [Parse_Cal(Val,Calibrate) for Val in Calibrate_Keys],
        axis=0,
        ignore_index=True)

    Calibrate_Summary = pd.concat(
        [Calibrate_Summary,Coef_Summary],
        axis=0)

    return(Calibrate_Summary)


    
def Parse_Metadata(config,key_list):
    MetaData = pd.concat(
        [pd.DataFrame(data={'Attribute':config[key].keys(),
                            'Value':config[key].values()}) for key in key_list],
                            axis=0,
                            ignore_index=True
                        )
    return (MetaData)



Path = '..\\BB\\Data_To_Process\\'

data_Means = ['CO2 Absorptance', 'H2O Absorptance',
'CO2 (mmol/m^3)', 'H2O (mmol/m^3)',
'Block Temperature (C)', 'Total Pressure (kPa)',
'Box Pressure (kPa)', 'Head Pressure (kPa)', 'Aux 1 - U (m/s)',
'Aux 2 - V (m/s)', 'Aux 3 - W (m/s)', 'Aux 4 - SOS (m/s)',
'Cooler Voltage (V)', 'Chopper Cooler Voltage (V)',
'Dew Point (C)',
'Cell Temperature (C)', 'Temperature In (C)',
'Temperature Out (C)', 'Average Signal Strength',
'Flow Rate (lpm)',
'Flow Pressure (kPa)', 'Flow Power (V)', 'Flow Drive (%)',
'CH4 (umol/mol)','CH4 Temperature', 'CH4 Pressure',
'CH4 Signal Strength']

status_Means = ['OPTICSTEMP', 'OPTICSRH']

data_Diagnostics = ['Diagnostic Value',
'Diagnostic Value 2', 'CH4 Diagnostic Value']

metadata_Tags = ['Site','Station','Timing','Instruments']

In [71]:



def Summarize_Data(Data,means,diagnostics=None):

    Data_Summary = Data[means].mean().to_frame().reset_index()
    Data_Summary.columns=['Attribute','Value']
    if diagnostics not None:
        Count = pd.DataFrame(data={'Attribute':'N_Samples',
        'Value':Data['Nanoseconds'].count()},index=[0])
        # Temporary implementation
        # Need to sort out more appropriate approach
        # Storing as an array of unique values may be better?
        data_Diagnostics = Data[diagnostics].mode().T.reset_index()
        data_Diagnostics.columns=['Attribute','Value']
        Data_Summary = pd.concat(
                            [Data_Summary,data_Diagnostics,Count],
                            axis=0,
                            ignore_index=True
                        
                            )
    return (Data_Summary)


config = configparser.ConfigParser()
def read_GHG(root,name):
    # Read the zipped data without extracting
    with zipfile.ZipFile(root+'\\'+name+'.ghg', 'r') as zip_ref:
        config.read_file(TextIOWrapper(zip_ref.open(name+'.metadata'), 'utf-8'))
        MetaData = Parse_Metadata(config,metadata_Tags)
        Data = pd.read_csv(zip_ref.open(name+'.data'),delimiter='\t',skiprows=7)
        Data_Summary = Summarize_Data(Data,data_Means,data_Diagnostics)
        Status_Summary = Summarize_Data(Data,data_Means,data_Diagnostics)
        # Status = pd.read_csv(zip_ref.open(name+'-li7700.status'),delimiter='\t',skiprows=7)
        Calibrations = (Read_co2app(zip_ref.open('system_config/co2app.conf').read().decode("utf-8")))
        
    Status_Summary = Status[status_Means].mean().to_frame().reset_index()
    Status_Summary.columns=['Attribute','Value']

    Summary = pd.concat(
        [MetaData,Data_Summary,Status_Summary,Calibrations],
        axis=0,
        ignore_index=True).set_index('Attribute').T
    # # Get the file timestamp
    # Summary['N_MetaData_Attributes']=Summary.columns.shape[0]
    TimeStamp = datetime.strptime(name.split('_')[0],'%Y-%m-%dT%H%M%S')
    # print(Summary)
    Summary['TimeStamp'] = [TimeStamp]
    Summary['DataPath'] = [root]
    Summary['FileName'] = [name]
    return(Summary)

T1 = time.time()
i = 0

print('Processing ghg files ')
f = FloatProgress(min=0, max=1) 
display(f) 
# Walk through the directory to find all "raw" folders
for (root, dirs, files) in sorted(os.walk(Path)):
    # .ghg files are located at the end of each directory tree
    if root.__contains__('raw') and len(dirs)==0:
        for file in files:
            name, tag = file.split('.')
            if tag == 'ghg':  
                # read info in .ghg files and add to a dataframe
                if i == 0:
                    Records = read_GHG(root,name).copy()
                    i += 1
                elif i <= 10:
                    Records = pd.concat(
                        [Records,read_GHG(root,name)],
                    axis=0,
                    ignore_index=True
                    )
                    i += 1
            f.value = i/len(files)

print('Processing time ', time.time()-T1) 
print('To inspect ', i, ' .ghg files')        
Records.head()


Processing ghg files 


FloatProgress(value=0.0, max=1.0)

                     Attribute         Value
0              CO2 Absorptance      0.119058
1              H2O Absorptance      0.060549
2               CO2 (mmol/m^3)     17.260056
3               H2O (mmol/m^3)    564.995190
4        Block Temperature (C)     19.686852
5         Total Pressure (kPa)     99.349337
6           Box Pressure (kPa)    100.447053
7          Head Pressure (kPa)     -1.097673
8              Aux 1 - U (m/s)     -1.048743
9              Aux 2 - V (m/s)     -2.700780
10             Aux 3 - W (m/s)      0.003336
11           Aux 4 - SOS (m/s)    341.871163
12          Cooler Voltage (V)      2.129143
13  Chopper Cooler Voltage (V)      1.435153
14               Dew Point (C)     11.636568
15        Cell Temperature (C)     19.686366
16          Temperature In (C)     19.674698
17         Temperature Out (C)     19.689283
18     Average Signal Strength     92.168732
19             Flow Rate (lpm)     16.449873
20         Flow Pressure (kPa)      1.484253
21        

Attribute,site_name,altitude,latitude,longitude,canopy_height,displacement_height,roughness_length,station_name,logger_id,logger_sw_version,...,H2O_SD1,H2O_SD2,H2O_SD3,Pressure_A0,Pressure_A1,MaxRef_B,MaxRef_C,TimeStamp,DataPath,FileName
0,BurnsBog2018,-7.4,49.1293869,-122.9848709,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 12:09:14,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T120914_LI-7200
1,BurnsBog2018,-7.4,49.1293869,-122.9848709,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 12:30:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T123000_LI-7200
2,BurnsBog2018,-6.7,49.12932968,-122.9848862,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 13:00:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T130000_LI-7200
3,BurnsBog2018,0.8,49.12933731,-122.9848938,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 13:30:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T133000_LI-7200
4,BurnsBog2018,0.8,49.12933731,-122.9848938,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 14:00:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T140000_LI-7200


These are all the values contained in the .METADATA, .DATA and .STATUS files.

## Metadata Values

More info [here](https://www.licor.com/env/support/EddyPro/topics/metadata-file-editor.html)

Site
['site_name', 'altitude', 'latitude', 'longitude', 'canopy_height', 'displacement_height', 'roughness_length']

Station
['station_name', 'logger_id', 'logger_sw_version']

Timing
['acquisition_frequency', 'file_duration']

Instruments
['instr_1_manufacturer', 'instr_1_model', 'instr_1_sn', 'instr_1_sw_version', 'instr_1_id', 'instr_1_height', 'instr_1_wformat', 'instr_1_wref', 'instr_1_north_offset', 'instr_1_head_corr', 'instr_1_northward_separation', 'instr_1_eastward_separation', 'instr_1_vertical_separation', 'instr_2_manufacturer', 'instr_2_model', 'instr_2_sn', 'instr_2_id', 'instr_2_sw_version', 'instr_2_tube_length', 'instr_2_tube_diameter', 'instr_2_tube_flowrate', 'instr_2_northward_separation', 'instr_2_eastward_separation', 'instr_2_vertical_separation', 'instr_3_manufacturer', 'instr_3_model', 'instr_3_sn', 'instr_3_id', 'instr_3_sw_version', 'instr_3_tube_length', 'instr_3_tube_diameter', 'instr_3_tube_flowrate', 'instr_3_northward_separation', 'instr_3_eastward_separation', 'instr_3_vertical_separation']

FileDescription
['separator', 'flag_discards_if_above', 'header_rows', 'data_label', 'col_1_variable', 'col_1_instrument', 'col_1_measure_type', 'col_1_unit_in', 'col_1_conversion', 'col_1_min_value', 'col_1_max_value', 'col_1_unit_out', 'col_1_a_value', 'col_1_b_value', 'col_1_nom_timelag', 'col_1_min_timelag', 'col_1_max_timelag', 'col_2_variable', 'col_2_instrument', 'col_2_measure_type', 'col_2_unit_in', 'col_2_conversion', 'col_2_min_value', 'col_2_max_value', 'col_2_unit_out', 'col_2_a_value', 'col_2_b_value', 'col_2_nom_timelag', 'col_2_min_timelag', 'col_2_max_timelag', 'col_3_variable', 'col_3_instrument', 'col_3_measure_type', 'col_3_unit_in', 'col_3_conversion', 'col_3_min_value', 'col_3_max_value', 'col_3_unit_out', 'col_3_a_value', 'col_3_b_value', 'col_3_nom_timelag', 'col_3_min_timelag', 'col_3_max_timelag', 'col_4_variable', 'col_4_instrument', 'col_4_measure_type', 'col_4_unit_in', 'col_4_conversion', 'col_4_min_value', 'col_4_max_value', 'col_4_unit_out', 'col_4_a_value', 'col_4_b_value', 'col_4_nom_timelag', 'col_4_min_timelag', 'col_4_max_timelag', 'col_5_variable', 'col_5_instrument', 'col_5_measure_type', 'col_5_unit_in', 'col_5_conversion', 'col_5_min_value', 'col_5_max_value', 'col_5_unit_out', 'col_5_a_value', 'col_5_b_value', 'col_5_nom_timelag', 'col_5_min_timelag', 'col_5_max_timelag', 'col_6_variable', 'col_6_instrument', 'col_6_measure_type', 'col_6_unit_in', 'col_6_conversion', 'col_6_min_value', 'col_6_max_value', 'col_6_unit_out', 'col_6_a_value', 'col_6_b_value', 'col_6_nom_timelag', 'col_6_min_timelag', 'col_6_max_timelag', 'col_7_variable', 'col_7_instrument', 'col_7_measure_type', 'col_7_unit_in', 'col_7_conversion', 'col_7_min_value', 'col_7_max_value', 'col_7_unit_out', 'col_7_a_value', 'col_7_b_value', 'col_7_nom_timelag', 'col_7_min_timelag', 'col_7_max_timelag', 'col_8_variable', 'col_8_instrument', 'col_8_measure_type', 'col_8_unit_in', 'col_8_conversion', 'col_8_min_value', 'col_8_max_value', 'col_8_unit_out', 'col_8_a_value', 'col_8_b_value', 'col_8_nom_timelag', 'col_8_min_timelag', 'col_8_max_timelag', 'col_9_variable', 'col_9_instrument', 'col_9_measure_type', 'col_9_unit_in', 'col_9_conversion', 'col_9_min_value', 'col_9_max_value', 'col_9_unit_out', 'col_9_a_value', 'col_9_b_value', 'col_9_nom_timelag', 'col_9_min_timelag', 'col_9_max_timelag', 'col_10_variable', 'col_10_instrument', 'col_10_measure_type', 'col_10_unit_in', 'col_10_conversion', 'col_10_min_value', 'col_10_max_value', 'col_10_unit_out', 'col_10_a_value', 'col_10_b_value', 'col_10_nom_timelag', 'col_10_min_timelag', 'col_10_max_timelag', 'col_11_variable', 'col_11_instrument', 'col_11_measure_type', 'col_11_unit_in', 'col_11_conversion', 'col_11_min_value', 'col_11_max_value', 'col_11_unit_out', 'col_11_a_value', 'col_11_b_value', 'col_11_nom_timelag', 'col_11_min_timelag', 'col_11_max_timelag', 'col_12_variable', 'col_12_instrument', 'col_12_measure_type', 'col_12_unit_in', 'col_12_conversion', 'col_12_min_value', 'col_12_max_value', 'col_12_unit_out', 'col_12_a_value', 'col_12_b_value', 'col_12_nom_timelag', 'col_12_min_timelag', 'col_12_max_timelag', 'col_13_variable', 'col_13_instrument', 'col_13_measure_type', 'col_13_unit_in', 'col_13_conversion', 'col_13_min_value', 'col_13_max_value', 'col_13_unit_out', 'col_13_a_value', 'col_13_b_value', 'col_13_nom_timelag', 'col_13_min_timelag', 'col_13_max_timelag', 'col_14_variable', 'col_14_instrument', 'col_14_measure_type', 'col_14_unit_in', 'col_14_conversion', 'col_14_min_value', 'col_14_max_value', 'col_14_unit_out', 'col_14_a_value', 'col_14_b_value', 'col_14_nom_timelag', 'col_14_min_timelag', 'col_14_max_timelag', 'col_15_variable', 'col_15_instrument', 'col_15_measure_type', 'col_15_unit_in', 'col_15_conversion', 'col_15_min_value', 'col_15_max_value', 'col_15_unit_out', 'col_15_a_value', 'col_15_b_value', 'col_15_nom_timelag', 'col_15_min_timelag', 'col_15_max_timelag', 'col_16_variable', 'col_16_instrument', 'col_16_measure_type', 'col_16_unit_in', 'col_16_conversion', 'col_16_min_value', 'col_16_max_value', 'col_16_unit_out', 'col_16_a_value', 'col_16_b_value', 'col_16_nom_timelag', 'col_16_min_timelag', 'col_16_max_timelag', 'col_17_variable', 'col_17_instrument', 'col_17_measure_type', 'col_17_unit_in', 'col_17_conversion', 'col_17_min_value', 'col_17_max_value', 'col_17_unit_out', 'col_17_a_value', 'col_17_b_value', 'col_17_nom_timelag', 'col_17_min_timelag', 'col_17_max_timelag', 'col_18_variable', 'col_18_instrument', 'col_18_measure_type', 'col_18_unit_in', 'col_18_conversion', 'col_18_min_value', 'col_18_max_value', 'col_18_unit_out', 'col_18_a_value', 'col_18_b_value', 'col_18_nom_timelag', 'col_18_min_timelag', 'col_18_max_timelag', 'col_19_variable', 'col_19_instrument', 'col_19_measure_type', 'col_19_unit_in', 'col_19_conversion', 'col_19_min_value', 'col_19_max_value', 'col_19_unit_out', 'col_19_a_value', 'col_19_b_value', 'col_19_nom_timelag', 'col_19_min_timelag', 'col_19_max_timelag', 'col_20_variable', 'col_20_instrument', 'col_20_measure_type', 'col_20_unit_in', 'col_20_conversion', 'col_20_min_value', 'col_20_max_value', 'col_20_unit_out', 'col_20_a_value', 'col_20_b_value', 'col_20_nom_timelag', 'col_20_min_timelag', 'col_20_max_timelag', 'col_21_variable', 'col_21_instrument', 'col_21_measure_type', 'col_21_unit_in', 'col_21_conversion', 'col_21_min_value', 'col_21_max_value', 'col_21_unit_out', 'col_21_a_value', 'col_21_b_value', 'col_21_nom_timelag', 'col_21_min_timelag', 'col_21_max_timelag', 'col_22_variable', 'col_22_instrument', 'col_22_measure_type', 'col_22_unit_in', 'col_22_conversion', 'col_22_min_value', 'col_22_max_value', 'col_22_unit_out', 'col_22_a_value', 'col_22_b_value', 'col_22_nom_timelag', 'col_22_min_timelag', 'col_22_max_timelag', 'col_23_variable', 'col_23_instrument', 'col_23_measure_type', 'col_23_unit_in', 'col_23_conversion', 'col_23_min_value', 'col_23_max_value', 'col_23_unit_out', 'col_23_a_value', 'col_23_b_value', 'col_23_nom_timelag', 'col_23_min_timelag', 'col_23_max_timelag', 'col_24_variable', 'col_24_instrument', 'col_24_measure_type', 'col_24_unit_in', 'col_24_conversion', 'col_24_min_value', 'col_24_max_value', 'col_24_unit_out', 'col_24_a_value', 'col_24_b_value', 'col_24_nom_timelag', 'col_24_min_timelag', 'col_24_max_timelag', 'col_25_variable', 'col_25_instrument', 'col_25_measure_type', 'col_25_unit_in', 'col_25_conversion', 'col_25_min_value', 'col_25_max_value', 'col_25_unit_out', 'col_25_a_value', 'col_25_b_value', 'col_25_nom_timelag', 'col_25_min_timelag', 'col_25_max_timelag', 'col_26_variable', 'col_26_instrument', 'col_26_measure_type', 'col_26_unit_in', 'col_26_conversion', 'col_26_min_value', 'col_26_max_value', 'col_26_unit_out', 'col_26_a_value', 'col_26_b_value', 'col_26_nom_timelag', 'col_26_min_timelag', 'col_26_max_timelag', 'col_27_variable', 'col_27_instrument', 'col_27_measure_type', 'col_27_unit_in', 'col_27_conversion', 'col_27_min_value', 'col_27_max_value', 'col_27_unit_out', 'col_27_a_value', 'col_27_b_value', 'col_27_nom_timelag', 'col_27_min_timelag', 'col_27_max_timelag', 'col_28_variable', 'col_28_instrument', 'col_28_measure_type', 'col_28_unit_in', 'col_28_conversion', 'col_28_min_value', 'col_28_max_value', 'col_28_unit_out', 'col_28_a_value', 'col_28_b_value', 'col_28_nom_timelag', 'col_28_min_timelag', 'col_28_max_timelag', 'col_29_variable', 'col_29_instrument', 'col_29_measure_type', 'col_29_unit_in', 'col_29_conversion', 'col_29_min_value', 'col_29_max_value', 'col_29_unit_out', 'col_29_a_value', 'col_29_b_value', 'col_29_nom_timelag', 'col_29_min_timelag', 'col_29_max_timelag', 'col_30_variable', 'col_30_instrument', 'col_30_measure_type', 'col_30_unit_in', 'col_30_conversion', 'col_30_min_value', 'col_30_max_value', 'col_30_unit_out', 'col_30_a_value', 'col_30_b_value', 'col_30_nom_timelag', 'col_30_min_timelag', 'col_30_max_timelag', 'col_31_variable', 'col_31_instrument', 'col_31_measure_type', 'col_31_unit_in', 'col_31_conversion', 'col_31_min_value', 'col_31_max_value', 'col_31_unit_out', 'col_31_a_value', 'col_31_b_value', 'col_31_nom_timelag', 'col_31_min_timelag', 'col_31_max_timelag', 'col_32_variable', 'col_32_instrument', 'col_32_measure_type', 'col_32_unit_in', 'col_32_conversion', 'col_32_min_value', 'col_32_max_value', 'col_32_unit_out', 'col_32_a_value', 'col_32_b_value', 'col_32_nom_timelag', 'col_32_min_timelag', 'col_32_max_timelag', 'col_33_variable', 'col_33_instrument', 'col_33_measure_type', 'col_33_unit_in', 'col_33_conversion', 'col_33_min_value', 'col_33_max_value', 'col_33_unit_out', 'col_33_a_value', 'col_33_b_value', 'col_33_nom_timelag', 'col_33_min_timelag', 'col_33_max_timelag', 'col_34_variable', 'col_34_instrument', 'col_34_measure_type', 'col_34_unit_in', 'col_34_conversion', 'col_34_min_value', 'col_34_max_value', 'col_34_unit_out', 'col_34_a_value', 'col_34_b_value', 'col_34_nom_timelag', 'col_34_min_timelag', 'col_34_max_timelag', 'col_35_variable', 'col_35_instrument', 'col_35_measure_type', 'col_35_unit_in', 'col_35_conversion', 'col_35_min_value', 'col_35_max_value', 'col_35_unit_out', 'col_35_a_value', 'col_35_b_value', 'col_35_nom_timelag', 'col_35_min_timelag', 'col_35_max_timelag', 'col_36_variable', 'col_36_instrument', 'col_36_measure_type', 'col_36_unit_in', 'col_36_conversion', 'col_36_min_value', 'col_36_max_value', 'col_36_unit_out', 'col_36_a_value', 'col_36_b_value', 'col_36_nom_timelag', 'col_36_min_timelag', 'col_36_max_timelag', 'col_37_variable', 'col_37_instrument', 'col_37_measure_type', 'col_37_unit_in', 'col_37_conversion', 'col_37_min_value', 'col_37_max_value', 'col_37_unit_out', 'col_37_a_value', 'col_37_b_value', 'col_37_nom_timelag', 'col_37_min_timelag', 'col_37_max_timelag', 'col_38_variable', 'col_38_instrument', 'col_38_measure_type', 'col_38_unit_in', 'col_38_conversion', 'col_38_min_value', 'col_38_max_value', 'col_38_unit_out', 'col_38_a_value', 'col_38_b_value', 'col_38_nom_timelag', 'col_38_min_timelag', 'col_38_max_timelag', 'col_39_variable', 'col_39_instrument', 'col_39_measure_type', 'col_39_unit_in', 'col_39_conversion', 'col_39_min_value', 'col_39_max_value', 'col_39_unit_out', 'col_39_a_value', 'col_39_b_value', 'col_39_nom_timelag', 'col_39_min_timelag', 'col_39_max_timelag', 'col_40_variable', 'col_40_instrument', 'col_40_measure_type', 'col_40_unit_in', 'col_40_conversion', 'col_40_min_value', 'col_40_max_value', 'col_40_unit_out', 'col_40_a_value', 'col_40_b_value', 'col_40_nom_timelag', 'col_40_min_timelag', 'col_40_max_timelag', 'col_41_variable', 'col_41_instrument', 'col_41_measure_type', 'col_41_unit_in', 'col_41_conversion', 'col_41_min_value', 'col_41_max_value', 'col_41_unit_out', 'col_41_a_value', 'col_41_b_value', 'col_41_nom_timelag', 'col_41_min_timelag', 'col_41_max_timelag', 'col_42_variable', 'col_42_instrument', 'col_42_measure_type', 'col_42_unit_in', 'col_42_conversion', 'col_42_min_value', 'col_42_max_value', 'col_42_unit_out', 'col_42_a_value', 'col_42_b_value', 'col_42_nom_timelag', 'col_42_min_timelag', 'col_42_max_timelag', 'col_43_variable', 'col_43_instrument', 'col_43_measure_type', 'col_43_unit_in', 'col_43_conversion', 'col_43_min_value', 'col_43_max_value', 'col_43_unit_out', 'col_43_a_value', 'col_43_b_value', 'col_43_nom_timelag', 'col_43_min_timelag', 'col_43_max_timelag', 'col_44_variable', 'col_44_instrument', 'col_44_measure_type', 'col_44_unit_in', 'col_44_conversion', 'col_44_min_value', 'col_44_max_value', 'col_44_unit_out', 'col_44_a_value', 'col_44_b_value', 'col_44_nom_timelag', 'col_44_min_timelag', 'col_44_max_timelag', 'col_45_variable', 'col_45_instrument', 'col_45_measure_type', 'col_45_unit_in', 'col_45_conversion', 'col_45_min_value', 'col_45_max_value', 'col_45_unit_out', 'col_45_a_value', 'col_45_b_value', 'col_45_nom_timelag', 'col_45_min_timelag', 'col_45_max_timelag', 'col_46_variable', 'col_46_instrument', 'col_46_measure_type', 'col_46_unit_in', 'col_46_conversion', 'col_46_min_value', 'col_46_max_value', 'col_46_unit_out', 'col_46_a_value', 'col_46_b_value', 'col_46_nom_timelag', 'col_46_min_timelag', 'col_46_max_timelag', 'col_47_variable', 'col_47_instrument', 'col_47_measure_type', 'col_47_unit_in', 'col_47_conversion', 'col_47_min_value', 'col_47_max_value', 'col_47_unit_out', 'col_47_a_value', 'col_47_b_value', 'col_47_nom_timelag', 'col_47_min_timelag', 'col_47_max_timelag', 'col_48_variable', 'col_48_instrument', 'col_48_measure_type', 'col_48_unit_in', 'col_48_conversion', 'col_48_min_value', 'col_48_max_value', 'col_48_unit_out', 'col_48_a_value', 'col_48_b_value', 'col_48_nom_timelag', 'col_48_min_timelag', 'col_48_max_timelag', 'col_49_variable', 'col_49_instrument', 'col_49_measure_type', 'col_49_unit_in', 'col_49_conversion', 'col_49_min_value', 'col_49_max_value', 'col_49_unit_out', 'col_49_a_value', 'col_49_b_value', 'col_49_nom_timelag', 'col_49_min_timelag', 'col_49_max_timelag', 'col_50_variable', 'col_50_instrument', 'col_50_measure_type', 'col_50_unit_in', 'col_50_conversion', 'col_50_min_value', 'col_50_max_value', 'col_50_unit_out', 'col_50_a_value', 'col_50_b_value', 'col_50_nom_timelag', 'col_50_min_timelag', 'col_50_max_timelag', 'col_51_variable', 'col_51_instrument', 'col_51_measure_type', 'col_51_unit_in', 'col_51_conversion', 'col_51_min_value', 'col_51_max_value', 'col_51_unit_out', 'col_51_a_value', 'col_51_b_value', 'col_51_nom_timelag', 'col_51_min_timelag', 'col_51_max_timelag', 'col_52_variable', 'col_52_instrument', 'col_52_measure_type', 'col_52_unit_in', 'col_52_conversion', 'col_52_min_value', 'col_52_max_value', 'col_52_unit_out', 'col_52_a_value', 'col_52_b_value', 'col_52_nom_timelag', 'col_52_min_timelag', 'col_52_max_timelag', 'col_53_variable', 'col_53_instrument', 'col_53_measure_type', 'col_53_unit_in', 'col_53_conversion', 'col_53_min_value', 'col_53_max_value', 'col_53_unit_out', 'col_53_a_value', 'col_53_b_value', 'col_53_nom_timelag', 'col_53_min_timelag', 'col_53_max_timelag', 'col_54_variable', 'col_54_instrument', 'col_54_measure_type', 'col_54_unit_in', 'col_54_conversion', 'col_54_min_value', 'col_54_max_value', 'col_54_unit_out', 'col_54_a_value', 'col_54_b_value', 'col_54_nom_timelag', 'col_54_min_timelag', 'col_54_max_timelag', 'col_55_variable', 'col_55_instrument', 'col_55_measure_type', 'col_55_unit_in', 'col_55_conversion', 'col_55_min_value', 'col_55_max_value', 'col_55_unit_out', 'col_55_a_value', 'col_55_b_value', 'col_55_nom_timelag', 'col_55_min_timelag', 'col_55_max_timelag', 'col_56_variable', 'col_56_instrument', 'col_56_measure_type', 'col_56_unit_in', 'col_56_conversion', 'col_56_min_value', 'col_56_max_value', 'col_56_unit_out', 'col_56_a_value', 'col_56_b_value', 'col_56_nom_timelag', 'col_56_min_timelag', 'col_56_max_timelag']

## Data Values
More info on the data file [here](https://www.licor.com/env/support/LI-7200RS/topics/data-files.html); info on 7200/CSAT diagnositcs [here](https://www.licor.com/env/support/LI-7200RS/topics/gas-analyzer-diagnostics.html) and [here](https://www.licor.com/env/support/LI-7200RS/topics/diagnostics-eddypro.html#Gas) and on the 7700 diagnostics [here](https://www.licor.com/env/support/LI-7700/topics/data-files.html).

'Seconds', 'Nanoseconds', 'Sequence Number', 'Diagnostic Value',
'Diagnostic Value 2', 'CO2 Absorptance', 'H2O Absorptance',
'CO2 (mmol/m^3)', 'CO2 (mg/m^3)', 'H2O (mmol/m^3)', 'H2O (g/m^3)',
'Block Temperature (C)', 'Total Pressure (kPa)',
'Box Pressure (kPa)', 'Head Pressure (kPa)', 'Aux 1 - U (m/s)',
'Aux 2 - V (m/s)', 'Aux 3 - W (m/s)', 'Aux 4 - SOS (m/s)',
'Cooler Voltage (V)', 'Chopper Cooler Voltage (V)',
'Vin SmartFlux (V)', 'CO2 (umol/mol)', 'CO2 dry(umol/mol)',
'H2O (mmol/mol)', 'H2O dry(mmol/mol)', 'Dew Point (C)',
'Cell Temperature (C)', 'Temperature In (C)',
'Temperature Out (C)', 'Average Signal Strength',
'CO2 Signal Strength', 'H2O Signal Strength',
'Delta Signal Strength', 'Flow Rate (slpm)', 'Flow Rate (lpm)',
'Flow Pressure (kPa)', 'Flow Power (V)', 'Flow Drive (%)',
'H2O Sample', 'H2O Reference', 'CO2 Sample', 'CO2 Reference',
'HIT Power (W)', 'Vin HIT (V)', 'CH4 Seconds', 'CH4 (umol/mol)',
'CH4 (mmol/m^3)', 'CH4 Temperature', 'CH4 Pressure',
'CH4 Signal Strength', 'CH4 Diagnostic Value', 'CH4 Drop Rate (%)',
'CHK'

## Status Values

More info [here](https://www.licor.com/env/support/LI-7700/topics/data-files.html)

'MSEC', 'SECONDS', 'NANOSECONDS', 'DIAG', 'RSSI', 'REFRSSI',
'LCTSETPT', 'LCTACTUAL', 'BCTSETPT', 'BCTACTUAL', 'CHASSISTEMP',
'OPTICSTEMP', 'OPTICSRH', 'AUXREFTEMP', 'MOTORSETPT',
'MOTORACTUAL', 'USB', 'USBCAPACITY', 'USBFREESPACE', 'REF', 'GND',
'OPTICSTDELTA', 'BOTTOMHEATERW', 'TOPHEATERW', 'CHK'


In [63]:
Records['Diag']

Attribute,site_name,altitude,latitude,longitude,canopy_height,displacement_height,roughness_length,station_name,logger_id,logger_sw_version,...,H2O_SD1,H2O_SD2,H2O_SD3,Pressure_A0,Pressure_A1,MaxRef_B,MaxRef_C,TimeStamp,DataPath,FileName
0,BurnsBog2018,-7.4,49.1293869,-122.9848709,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 12:09:14,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T120914_LI-7200
1,BurnsBog2018,-7.4,49.1293869,-122.9848709,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 12:30:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T123000_LI-7200
2,BurnsBog2018,-6.7,49.12932968,-122.9848862,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 13:00:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T130000_LI-7200
3,BurnsBog2018,0.8,49.12933731,-122.9848938,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 13:30:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T133000_LI-7200
4,BurnsBog2018,0.8,49.12933731,-122.9848938,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 14:00:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T140000_LI-7200
5,BurnsBog2018,-2.9,49.12934113,-122.9848938,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 14:30:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T143000_LI-7200
6,BurnsBog2018,-2.9,49.12934113,-122.9848938,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 15:00:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T150000_LI-7200
7,BurnsBog2018,2.8,49.12932968,-122.9848785,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 15:30:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T153000_LI-7200
8,BurnsBog2018,7.5,49.12934875,-122.9848251,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 16:00:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T160000_LI-7200
9,BurnsBog2018,7.5,49.12934875,-122.9848251,.3,0,0.056,GeogFlux,LI-7200,8.9.0,...,1.910E-2,1.323E+0,2.385E+0,57.704,15.424,0.050,0.051,2022-06-17 16:30:00,..\BB\Data_To_Process\20220728\raw\2022\06,2022-06-17T163000_LI-7200
